<h1>참고 자료</h1>
<table>
    <thead>
        <tr>
            <th>구분</th>
            <th>제목</th>
            <th>내용</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>1</td>
            <td>NUMPY로 CCTY 인접 좌표 찾기</td>
            <td>https://parkgihyeon.github.io/project/cctv-api</td>
        </tr>
        <tr>
            <td>2</td>
            <td>KDtree 설명 -1</td>
            <td>https://m.blog.naver.com/laonple/221207919855</td>
        </tr>
        <tr>
            <td>3</td>
            <td>sklearn 에서 제공하는 KDTree</td>
            <td>https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html</td>
        </tr>
    </tbody>
</table>

<h1>프로그램 목표</h1>

<h2>관측 자료를 모델 자료에 매칭</h2>
<p>KDTree객체 생성 후 가장 가까운 점 탐색</p>


<h2>진행 순서</h2>
<h3> 알고리즘 비교</h3>
<p> 전체 데이터가 아닌 일부분으로 진행<p>
<ol>
    <li>데이터 로드 (1.12)</li>
    <li>데이터 설명 (EDA) (1.12)</li>
    <li>알고리즘 비교용 데이터 축소 (1.12)</li>
    <li>KDTree 알고리즘 (1.15)</li>
    <li>KDTree 알고리즘 - 전체 데이터에 적용 (1.19)</li>
    <li>RMSE,MSE 산출 (1.19)</li>
    <li>동일 매칭 조건으로 모델 자료 전체 병합</li>
</ol>

# 데이터 로드

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

from sklearn.neighbors import KDTree

import os
import glob

c:\Users\user\anaconda3\envs\DT2023\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# 파일내 모든 csv 파일을 읽어오는 함수(하위 폴더의 csv 까지 모두 불러옴)
def explore_files(directory):
    dfs = []  # 읽은 DataFrame을 저장할 리스트
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):  # 확장자가 .csv인 파일인지 확인
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path, low_memory=False)  # CSV 파일 읽기
                dfs.append(df)
    
    for subdir in dirs:
        subdir_path = os.path.join(root, subdir)
        dfs.extend(explore_files(subdir_path))  # 재귀적으로 하위 디렉토리 탐색하여 DataFrame 추가
    dfs_df = pd.concat(dfs, ignore_index=True) 
    return dfs_df

In [3]:
# 사용할 관측 데이터
data_obs_init = pd.read_csv(r'Y:\DATA\2023_SC2_ROAD_OBS\DATA_OBS\202301\202301_road_obs_all_KST.csv', low_memory=False)

In [4]:
data_obs_init.columns

Index(['timestamp', 'longitude', 'latitude', 'temperature', 'humidity',
       'road.temperature', 'surface.state', 'SFS_txt', 'waterlayer',
       'snowlayer', 'friction', 'dewpoint.temperature', 'filename', 'year',
       'month', 'day', 'hour', 'min', 'sec', 'ym', 'ymd', 'ymdh', 'loc',
       'roadname', 'dir', 'sensor'],
      dtype='object')

In [5]:
#데이터의 timestmp는 센서에 따라서 포멧이 다른 경우가 있기 때문에 각각 int로 저장해둔 시간 변수를 사용
data_obs_1 = data_obs_init[['longitude', 'latitude', 'temperature', 'humidity',
                             'road.temperature', 'surface.state', 'SFS_txt', 'waterlayer',
                             'snowlayer', 'friction', 'dewpoint.temperature', 'filename', 'year',
                             'month', 'day', 'hour', 'min', 'sec',  'loc','roadname', 'dir', 'sensor']]

In [6]:
data_obs_1

,longitude,latitude,temperature,humidity,road.temperature,surface.state,SFS_txt,waterlayer,snowlayer,friction,...,year,month,day,hour,min,sec,loc,roadname,dir,sensor
0,127.113716,37.556753,5.5200,39.880001,7.190002,SurfaceState.DRY,DRY,0.011927,0.0,0.819894,...,2023,1,19,15,6,31.0,seoul,gangbyeon,U,vaisala
1,127.113711,37.556733,5.5200,39.880001,7.269989,SurfaceState.DRY,DRY,0.011861,0.0,0.819898,...,2023,1,19,15,6,31.0,seoul,gangbyeon,U,vaisala
2,127.113707,37.556716,5.5200,39.880001,7.269989,SurfaceState.DRY,DRY,0.011903,0.0,0.819896,...,2023,1,19,15,6,31.0,seoul,gangbyeon,U,vaisala
3,127.113704,37.556703,5.5200,39.880001,7.269989,SurfaceState.DRY,DRY,0.012149,0.0,0.819881,...,2023,1,19,15,6,31.0,seoul,gangbyeon,U,vaisala
4,127.113699,37.556683,5.5200,39.880001,7.190002,SurfaceState.DRY,DRY,0.012074,0.0,0.819886,...,2023,1,19,15,6,31.0,seoul,gangbyeon,U,vaisala
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214748,126.751067,37.390217,3.1373,77.696000,0.711200,5,chemically-wet,0.045700,0.0,0.807600,...,2023,1,20,1,23,16.0,sihueng,SHR4,U,lufft
214749,126.751067,37.390217,3.1373,77.696000,0.715500,5,chemically-wet,0.046400,0.0,0.807500,...,2023,1,20,1,23,16.0,sihueng,SHR4,U,lufft
214750,126.751067,37.390217,3.1284,77.670600,0.732800,5,chemically-wet,0.046800,0.0,0.807400,...,2023,1,20,1,23,16.0,sihueng,SHR4,U,lufft
214751,126.751067,37.390217,3.1284,77.670600,0.696100,5,chemically-wet,0.047100,0.0,0.807300,...,2023,1,20,1,23,16.0,sihueng,SHR4,U,lufft


## roadname = gangbyeon , dir = u 데이터로 테스트

In [7]:
cond1 = data_obs_1['roadname'] == 'gangbyeon'
cond2 = data_obs_1['dir'] == 'U'
data_obs_test = data_obs_1[cond1&cond2]

In [8]:
print(data_obs_test['month'].unique())
print(data_obs_test['day'].unique())
print(data_obs_test['hour'].unique())
# 1월 19일 15시 데이터로 진행 > 관측자료

print("관측 자료 개수",len(data_obs_test))

[1]
[19]
[15]
관측 자료 개수 31977


모델 자료는 <br>
jr mg org ss yc 각각 지역 바탕으로 만든 자료가 있음 각각 비교가 필요

|Y:/DATA/2023_SC2_ROAD_OBS/MODEL/seoul<br>
|<br>
<pre>|----- jr|</pre>
<pre>         |-----0.PRE </pre>
<pre>         |-----1.OUT |</pre>
<pre>                     |-----202302</pre>
<pre>                     |-----202301</pre>
<pre>                     |-----202212 |</pre>
<pre>                                  |-----월별 파일 아래에 일별로 정리 되어있음</pre>

In [9]:
model_total_list = []

In [10]:
model_sites = ['jr','mg','org','ss','yc']
yyyymm = '202301'
dd_obs = 19
hour = 15
days = [f'{dd_obs-1}',f'{dd_obs}']
for model_site in model_sites:
    for dd in days:
        model_file_dir = f'Y:/DATA/2023_SC2_ROAD_OBS/MODEL/seoul/{model_site}/1.OUT/{yyyymm}/{dd}/'
        csv_files = glob.glob(f'{model_file_dir}/*.csv')
        site_list = []
        # 찾은 CSV 파일들을 출력
        for csv_file in csv_files:
            site = pd.read_csv(csv_file)
            site['site'] = f'{model_site}'
            site_list.append(site)
        site_df = pd.concat(site_list, ignore_index=True)
    model_total_list.append(site_df)


In [11]:
site_number = 1
model_total_list[site_number]['date_time'] = pd.to_datetime(model_total_list[0]['date_time'])
model_total_list[site_number]['hour'] = model_total_list[site_number]['date_time'].dt.hour
model_total_list[site_number]['day'] = model_total_list[site_number]['date_time'].dt.day
model = model_total_list[site_number]
model_1 = model[(model['hour'] == hour) & (model['day'] == dd_obs)]

In [12]:
model_cond1 = model_1['road_name'] == 'gangbyeon'
model_cond2 = model_1['direction'] == 'URL'
model_2 = model_1[model_cond1&model_cond2]

In [13]:
model_2

,date_time,update_time,loc,lon,lat,seq,p_hour,road_name,direction,altitude,link_id,road_temp,road_hydro,road_ice,site,hour,day
0,2023-01-19 15:00:00,2023-01-19 15:00:00,seoul,127.113670,37.556764,1,0,gangbyeon,URL,51.0,1040004101,15.717401,0,0,mg,15,19
1,2023-01-19 15:00:00,2023-01-19 15:00:00,seoul,127.113651,37.556681,2,0,gangbyeon,URL,52.0,1040004101,15.717142,0,0,mg,15,19
2,2023-01-19 15:00:00,2023-01-19 15:00:00,seoul,127.113633,37.556598,3,0,gangbyeon,URL,52.0,1040004101,15.694023,0,0,mg,15,19
3,2023-01-19 15:00:00,2023-01-19 15:00:00,seoul,127.113615,37.556520,4,0,gangbyeon,URL,53.0,1040004101,15.525951,0,0,mg,15,19
4,2023-01-19 15:00:00,2023-01-19 15:00:00,seoul,127.113598,37.556442,5,0,gangbyeon,URL,53.0,1040004101,15.526000,0,0,mg,15,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3507,2023-01-19 15:00:00,2023-01-19 15:00:00,seoul,126.863471,37.576283,3508,0,gangbyeon,URL,35.0,1130007300,15.693569,0,0,mg,15,19
3508,2023-01-19 15:00:00,2023-01-19 15:00:00,seoul,126.863395,37.576327,3509,0,gangbyeon,URL,35.0,1130007300,15.693569,0,0,mg,15,19
3509,2023-01-19 15:00:00,2023-01-19 15:00:00,seoul,126.863317,37.576368,3510,0,gangbyeon,URL,36.0,1130007300,15.693569,0,0,mg,15,19
3510,2023-01-19 15:00:00,2023-01-19 15:00:00,seoul,126.863239,37.576409,3511,0,gangbyeon,URL,36.0,1130007300,15.693569,0,0,mg,15,19



<h3>관측방향</h3>
<table>
    <tr>
        <td>URL</td>
        <td>강변,내부,올림픽, SHR4</td>
    </tr>
    <tr>
        <td>UR</td>
        <td>동부, SHR2</td>
    </tr>
    <tr>
        <td>DL</td>
        <td>서부, SHR1, SHR3</td>
    </tr>
<table>


<p>
    동일 기간 동일 위치에 대해서 관측 자료는 31,977 개 모델 자료는 3588개 (강변 URL 기준)<br />
    따라서 모델 자료에 근접하는 관측 자료를 선별하여 자료를 구성해야함
</p>

## KDTree를 이용한 모델 검증 프로그램

### KDTree 란
<pre>KDTree는 이진 트리로. 모든 트리의 노드가 k차원으로 이루어져있다.
공간을 기반으로 partitioning 하는 이진 트리, 점들을  k차원 공간에서 정리한다.
<a hrf = "https://haamjamie.tistory.com/31">KDTree의 쉬운 설명 </a>
<a hrf = "https://en.wikipedia.org/wiki/K-d_tree">KDTree-위키피디아 </a>

</pre>
<p>KD Tree의 장점</p>
<ul>
    <li> 빠른 최근접 이웃 검색: KD 트리는 최근접 이웃 검색에 특히 효과적입니다. 차원을 효과적으로 나누어 검색 영역을 줄이기 때문에 빠른 검색이 가능합니다.</li>
    <li>메모리 효율적인 데이터 구조: KD 트리는 데이터를 재귀적으로 나누는 방식으로 효율적인 메모리 구조를 가집니다. 이는 특히 대규모 데이터셋에서 중요합니다.</li>
    <li>유연성: KD 트리는 다양한 차원에서도 사용될 수 있습니다. 고차원 데이터에서도 효과적으로 작동하는 것이 주요 특징 중 하나입니다.</li>
    <li>효율적인 범위 검색: KD 트리는 데이터를 나누고 정렬함으로써 효율적인 범위 검색을 제공합니다. 이는 특정 영역에 속하는 데이터를 효과적으로 찾을 수 있도록 도와줍니다.</li>
    <li>균형 트리 구조: KD 트리는 균형 트리의 형태를 가지므로 검색 작업에서 최악의 경우 시나리오에 대한 안정성을 제공합니다. 이는 검색 시간을 일정 수준으로 유지하는 데 도움이 됩니다.</li>
</ul>

In [14]:
# roadname = gangbyeon , dir = u 데이터로 테스트
# 1월 19일 15시
model_data_df = model_2 # 해당 조건의 모델 데이터 
obs_date_df = data_obs_test # 해당 조건의 관측 데이터

<pre>모델의 위경도 컬럼명 : lon lat
관측 자료의 위경도 컬럼명 : longitude latitude
</pre>

### KDTree기반 알고리즘

In [15]:
#KDTree 방법
road_df = pd.DataFrame()
# 첫 번째 줄의 위경도 데이터
observ_line = np.array(list(zip(data_obs_test['longitude'], data_obs_test['latitude'])))

# 두 번째 줄의 위경도 데이터
model_line = np.array(list(zip(model_data_df['lon'], model_data_df['lat'])))

# KDTree 객체 생성
tree = KDTree(observ_line)

# 각 점마다 가장 가까운 점을 찾아 매칭
matched_points = []
for point in model_line:
    _, index = tree.query([point], k=1)  # k=1로 설정하여 가장 가까운 점 하나만 선택
    matched_points.append(observ_line[index[0]])

matched_lon = [point[0][0] for point in matched_points]
matched_lat = [point[0][1] for point in matched_points]

df = pd.DataFrame({'lon': model_line[:, 0], 'lat': model_line[:, 1], 'longitude': matched_lon, 'latitude': matched_lat})

days = model_data_df['day'].unique()

for day in days:

    model_date_2 = model_data_df[model_data_df['day'] == day]

    total_df = pd.merge(df,data_obs_test,on=['longitude','latitude'])
    total_df = pd.merge(total_df,model_date_2,on=['lon','lat'])
    total_df = total_df.drop_duplicates()
    road_df = pd.concat([road_df,total_df])

In [16]:
road_df.drop_duplicates(subset=['lon', 'lat'])

,lon,lat,longitude,latitude,temperature,humidity,road.temperature,surface.state,SFS_txt,waterlayer,...,road_name,direction,altitude,link_id,road_temp,road_hydro,road_ice,site,hour_y,day_y
0,127.113670,37.556764,127.113716,37.556753,5.52,39.880001,7.190002,SurfaceState.DRY,DRY,0.011927,...,gangbyeon,URL,51.0,1040004101,15.717401,0,0,mg,15,19
1,127.113651,37.556681,127.113696,37.556670,5.52,39.880001,7.190002,SurfaceState.DRY,DRY,0.012203,...,gangbyeon,URL,52.0,1040004101,15.717142,0,0,mg,15,19
2,127.113633,37.556598,127.113675,37.556587,5.52,39.880001,7.149994,SurfaceState.DRY,DRY,0.011418,...,gangbyeon,URL,52.0,1040004101,15.694023,0,0,mg,15,19
3,127.113615,37.556520,127.113655,37.556502,5.51,39.880001,7.010010,SurfaceState.DRY,DRY,0.005611,...,gangbyeon,URL,53.0,1040004101,15.525951,0,0,mg,15,19
4,127.113598,37.556442,127.113636,37.556428,5.51,39.880001,7.010010,SurfaceState.DRY,DRY,0.003610,...,gangbyeon,URL,53.0,1040004101,15.526000,0,0,mg,15,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3583,126.863471,37.576283,126.863494,37.576314,6.31,35.240002,8.190002,SurfaceState.DRY,DRY,0.005559,...,gangbyeon,URL,35.0,1130007300,15.693569,0,0,mg,15,19
3584,126.863395,37.576327,126.863420,37.576360,6.31,35.240002,8.290009,SurfaceState.DRY,DRY,0.007903,...,gangbyeon,URL,35.0,1130007300,15.693569,0,0,mg,15,19
3585,126.863317,37.576368,126.863343,37.576410,6.31,35.240002,8.369995,SurfaceState.DRY,DRY,0.008954,...,gangbyeon,URL,36.0,1130007300,15.693569,0,0,mg,15,19
3586,126.863239,37.576409,126.863263,37.576457,6.31,35.230000,8.410004,SurfaceState.DRY,DRY,0.001300,...,gangbyeon,URL,36.0,1130007300,15.693569,0,0,mg,15,19


## 전체 데이터에 적용
<p>데이터 모음부터 자료 생성까지</p>

In [17]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from sklearn.neighbors import KDTree

import os
import glob

# 함수로 기능을 정의
def point_match(roadname,dir,direction,site,model_site):

    data_obs_init = pd.read_csv(r'Y:\DATA\2023_SC2_ROAD_OBS\DATA_OBS\202301\202301_road_obs_all_KST.csv', low_memory=False)
    #데이터의 timestmp는 센서에 따라서 다른 경우가 있기 때문에 각각 int로 저장해둔 시간 변수를 사용
    data_obs_1 = data_obs_init[['longitude', 'latitude', 'temperature', 'humidity',
                                'road.temperature', 'surface.state', 'SFS_txt', 'waterlayer',
                                'snowlayer', 'friction', 'dewpoint.temperature', 'filename', 'year',
                                'month', 'day', 'hour', 'min', 'sec',  'loc','roadname', 'dir', 'sensor']]

    cond1 = data_obs_1['roadname'] == roadname
    cond2 = data_obs_1['dir'] == dir
    data_obs_test = data_obs_1[cond1&cond2]
    print('해당 관측 자료에 포함되는 년',data_obs_test['year'].unique())
    print('해당 관측 자료에 포함되는 월',data_obs_test['month'].unique())
    print('해당 관측 자료에 포함되는 일',data_obs_test['day'].unique())
    print('해당 관측 자료에 포함되는 시간',data_obs_test['hour'].unique())
    # 1월 19일 15시 데이터로 진행 > 관측자료
    print("관측 자료 개수",len(data_obs_test))
    year = data_obs_test['year'].unique()[0]
    month = data_obs_test['month'].unique()[0]
    day = data_obs_test['day'].unique()[0]
    hour = data_obs_test['hour'].unique()[0]

    yyyymm = f'{year}{month:02d}'
    dd_obs = day
    hour = hour
    days = [dd_obs-1,dd_obs] # 모델 자료 시간을 KST로 변환으로 인해 한 날짜의 데이터가 2개의 파일에 분할되어 있음

    for dd in days:
        model_file_dir = f'Y:/DATA/2023_SC2_ROAD_OBS/MODEL/{site}/{model_site}/1.OUT/{yyyymm}/{dd}/'
        csv_files = glob.glob(f'{model_file_dir}/*.csv')
        site_list = []
        # 찾은 CSV 파일들을 출력
        for csv_file in csv_files:
            model_road = pd.read_csv(csv_file)
            model_road['site'] = f'{model_site}'
            site_list.append(model_road)
        site_df = pd.concat(site_list, ignore_index=True)


    site_df['date_time'] = pd.to_datetime(site_df['date_time'])
    site_df['hour'] =site_df['date_time'].dt.hour
    site_df['day'] = site_df['date_time'].dt.day
    model = site_df
    model_1 = model[(model['hour'] == hour) & (model['day'] == dd_obs)]

    model_cond1 = model_1['road_name'] == roadname 
    model_cond2 = model_1['direction'] == direction
    model_2 = model_1[model_cond1&model_cond2]

    model_data_df = model_2.copy() # 해당 조건의 모델 데이터 / 재사용을 위해 복사본 사용
    obs_date_df = data_obs_test.copy() # 해당 조건의 관측 데이터 / 재사용을 위해 복사본 사용
    
    print('모델 데이터의 컬럼 : ',model_data_df.columns)
    print('관측 데이터의 컬럼 : ',obs_date_df.columns)
    #KDTree 를 통한 근접 점 찾기
    road_df = pd.DataFrame()
    # 첫 번째 줄의 위경도 데이터
    observ_line = np.array(list(zip(obs_date_df['longitude'], obs_date_df['latitude'])))

    # 두 번째 줄의 위경도 데이터
    model_line = np.array(list(zip(model_data_df['lon'], model_data_df['lat'])))

    # KDTree 객체 생성
    tree = KDTree(observ_line)

    # 각 점마다 가장 가까운 점을 찾아 매칭
    matched_points = []
    for point in model_line:
        _, index = tree.query([point], k=1)  # k=1로 설정하여 가장 가까운 점 하나만 선택
        matched_points.append(observ_line[index[0]])

    matched_lon = [point[0][0] for point in matched_points]
    matched_lat = [point[0][1] for point in matched_points]

    df = pd.DataFrame({'lon': model_line[:, 0], 'lat': model_line[:, 1], 'longitude': matched_lon, 'latitude': matched_lat})

    days = model_data_df['day'].unique()

    for day in days:

        model_date_2 = model_data_df[model_data_df['day'] == day]

        total_df = pd.merge(df,data_obs_test,on=['longitude','latitude'])
        total_df = pd.merge(total_df,model_date_2,on=['lon','lat','hour','day'])
        total_df = total_df.drop_duplicates()
        road_df = pd.concat([road_df,total_df])

    val_data = road_df.drop_duplicates(subset=['lon', 'lat'])


    return val_data

In [18]:
# 자료별 이름 방향1 방향2 서울/시흥 
# road_name, dir , direction, site
road_data = {
    1: ['gangbyeon', 'U', 'URL', 'seoul'],
    2: ['naebu', 'U', 'URL', 'seoul'],
    3: ['dongbu', 'U', 'UR', 'seoul'],
    4: ['dongbu', 'D', 'DL', 'seoul'],
    5: ['naebu', 'U', 'URL', 'seoul'],
    6: ['olympic', 'U', 'URL', 'seoul'],
    7: ['olympic', 'D', 'DLR', 'seoul'],
    8: ['seobu', 'D', 'DL', 'seoul'],
    9: ['SHR1', 'D', 'DL','sihueng'],
    10: ['SHR2', 'U', 'UR','sihueng'],
    11: ['SHR3', 'D', 'DL','sihueng'],
    12: ['SHR4', 'U', 'UR','sihueng']
}

# road_data[1] ->  ['gangbyeon', 'U', 'URL', 'seoul']
# road_data[1][0] -> 'gangbyeon'

In [19]:
data_match = point_match(road_data[1][0],road_data[1][1],road_data[1][2],road_data[1][3], 'org') # 위에서 정의한 point_match 함수로 원하는 관측 자료와 모델 자료를 매칭

해당 관측 자료에 포함되는 년 [2023]
해당 관측 자료에 포함되는 월 [1]
해당 관측 자료에 포함되는 일 [19]
해당 관측 자료에 포함되는 시간 [15]
관측 자료 개수 31977
모델 데이터의 컬럼 :  Index(['date_time', 'update_time', 'loc', 'lon', 'lat', 'seq', 'p_hour',
       'road_name', 'direction', 'altitude', 'link_id', 'road_temp',
       'road_hydro', 'road_ice', 'site', 'hour', 'day'],
      dtype='object')
관측 데이터의 컬럼 :  Index(['longitude', 'latitude', 'temperature', 'humidity', 'road.temperature',
       'surface.state', 'SFS_txt', 'waterlayer', 'snowlayer', 'friction',
       'dewpoint.temperature', 'filename', 'year', 'month', 'day', 'hour',
       'min', 'sec', 'loc', 'roadname', 'dir', 'sensor'],
      dtype='object')


In [20]:
data_match
#data_match.to_csv("원하는경로/원하는파일이름.csv", index=False) #csv 파일로 저장하기

,lon,lat,longitude,latitude,temperature,humidity,road.temperature,surface.state,SFS_txt,waterlayer,...,seq,p_hour,road_name,direction,altitude,link_id,road_temp,road_hydro,road_ice,site
0,127.113670,37.556764,127.113716,37.556753,5.52,39.880001,7.190002,SurfaceState.DRY,DRY,0.011927,...,1,0,gangbyeon,URL,51.0,1040004101,8.863161,0,0,org
1,127.113651,37.556681,127.113696,37.556670,5.52,39.880001,7.190002,SurfaceState.DRY,DRY,0.012203,...,2,0,gangbyeon,URL,52.0,1040004101,8.881300,0,0,org
2,127.113633,37.556598,127.113675,37.556587,5.52,39.880001,7.149994,SurfaceState.DRY,DRY,0.011418,...,3,0,gangbyeon,URL,52.0,1040004101,8.917889,0,0,org
3,127.113615,37.556520,127.113655,37.556502,5.51,39.880001,7.010010,SurfaceState.DRY,DRY,0.005611,...,4,0,gangbyeon,URL,53.0,1040004101,8.894870,0,0,org
4,127.113598,37.556442,127.113636,37.556428,5.51,39.880001,7.010010,SurfaceState.DRY,DRY,0.003610,...,5,0,gangbyeon,URL,53.0,1040004101,8.891675,0,0,org
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3583,126.863471,37.576283,126.863494,37.576314,6.31,35.240002,8.190002,SurfaceState.DRY,DRY,0.005559,...,3508,0,gangbyeon,URL,35.0,1130007300,9.366783,0,0,org
3584,126.863395,37.576327,126.863420,37.576360,6.31,35.240002,8.290009,SurfaceState.DRY,DRY,0.007903,...,3509,0,gangbyeon,URL,35.0,1130007300,9.366783,0,0,org
3585,126.863317,37.576368,126.863343,37.576410,6.31,35.240002,8.369995,SurfaceState.DRY,DRY,0.008954,...,3510,0,gangbyeon,URL,36.0,1130007300,9.366783,0,0,org
3586,126.863239,37.576409,126.863263,37.576457,6.31,35.230000,8.410004,SurfaceState.DRY,DRY,0.001300,...,3511,0,gangbyeon,URL,36.0,1130007300,9.366783,0,0,org


## RMSE 값 계산하기

In [21]:
from sklearn.metrics import mean_squared_error 
from math import sqrt

actual_values = data_match['road.temperature']
predicted_values = data_match['road_temp']
# RMSE 계산
rmse = sqrt(mean_squared_error(actual_values, predicted_values))
print(road_data[1][0],road_data[1][1],road_data[1][2],road_data[1][3], 'org')
print(f'RMSE between road.temperature and road_temp: {rmse}')

gangbyeon U URL seoul org
RMSE between road.temperature and road_temp: 1.4257524923550347


### 동일한 기준으로 여러개의 모델 자료 붙이기

In [22]:
data_match

,lon,lat,longitude,latitude,temperature,humidity,road.temperature,surface.state,SFS_txt,waterlayer,...,seq,p_hour,road_name,direction,altitude,link_id,road_temp,road_hydro,road_ice,site
0,127.113670,37.556764,127.113716,37.556753,5.52,39.880001,7.190002,SurfaceState.DRY,DRY,0.011927,...,1,0,gangbyeon,URL,51.0,1040004101,8.863161,0,0,org
1,127.113651,37.556681,127.113696,37.556670,5.52,39.880001,7.190002,SurfaceState.DRY,DRY,0.012203,...,2,0,gangbyeon,URL,52.0,1040004101,8.881300,0,0,org
2,127.113633,37.556598,127.113675,37.556587,5.52,39.880001,7.149994,SurfaceState.DRY,DRY,0.011418,...,3,0,gangbyeon,URL,52.0,1040004101,8.917889,0,0,org
3,127.113615,37.556520,127.113655,37.556502,5.51,39.880001,7.010010,SurfaceState.DRY,DRY,0.005611,...,4,0,gangbyeon,URL,53.0,1040004101,8.894870,0,0,org
4,127.113598,37.556442,127.113636,37.556428,5.51,39.880001,7.010010,SurfaceState.DRY,DRY,0.003610,...,5,0,gangbyeon,URL,53.0,1040004101,8.891675,0,0,org
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3583,126.863471,37.576283,126.863494,37.576314,6.31,35.240002,8.190002,SurfaceState.DRY,DRY,0.005559,...,3508,0,gangbyeon,URL,35.0,1130007300,9.366783,0,0,org
3584,126.863395,37.576327,126.863420,37.576360,6.31,35.240002,8.290009,SurfaceState.DRY,DRY,0.007903,...,3509,0,gangbyeon,URL,35.0,1130007300,9.366783,0,0,org
3585,126.863317,37.576368,126.863343,37.576410,6.31,35.240002,8.369995,SurfaceState.DRY,DRY,0.008954,...,3510,0,gangbyeon,URL,36.0,1130007300,9.366783,0,0,org
3586,126.863239,37.576409,126.863263,37.576457,6.31,35.230000,8.410004,SurfaceState.DRY,DRY,0.001300,...,3511,0,gangbyeon,URL,36.0,1130007300,9.366783,0,0,org


In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from sklearn.neighbors import KDTree

import os
import glob
# ver2

# 사용할 관측 데이터

# 디렉토리 내의 모든 CSV 파일 목록 가져오기
csv_files = glob.glob('./DATA/20230705_스시2_서울&시흥_1월관측_후처리자료/' + '*.csv')
print("CSV 파일 목록:")

for file in csv_files:

    print(file)
    obs_df = pd.read_csv(file, low_memory=False)
    obs_df.columns = obs_df.columns.str.lower()

    file_info = file.split('_')

    road_name = file_info[-4]
    date = file_info[-2]
    road_dir_1 = file_info[-1]
    road_dir_1 = road_dir_1.split('.')[0]
    date = file_info[-2]
    print(road_name, date, road_dir_1)

    data_obs_test = obs_df
    #데이터의 timestmp는 센서에 따라서 다른 경우가 있기 때문에 각각 int로 저장해둔 시간 변수를 사용
    data_obs_test["timestamp"] = pd.to_datetime(data_obs_test["timestamp"])
    data_obs_test["year"] = data_obs_test["timestamp"].dt.year
    data_obs_test["month"] = data_obs_test["timestamp"].dt.month
    data_obs_test["day"] = data_obs_test["timestamp"].dt.day
    data_obs_test["hour"] = data_obs_test["timestamp"].dt.hour
    print('해당 관측 자료에 포함되는 일',data_obs_test['day'].unique())
    print('해당 관측 자료에 포함되는 시간',data_obs_test['hour'].unique())
    # 1월 19일 15시 데이터로 진행 > 관측자료
    print("관측 자료 개수",len(data_obs_test))

    day = data_obs_test['day'].unique()[0]
    hour = data_obs_test['hour'].unique()[0]
    year = data_obs_test['year'].unique()[0]
    month = data_obs_test['month'].unique()[0]

    yyyymm = f'{year}{month:02d}'
    dd_obs = day
    hour = hour
    days = [dd_obs-1,dd_obs] # 모델 자료 시간을 KST로 변환으로 인해 한 날짜의 데이터가 2개의 파일에 분할되어 있음
    models = {'jr', 'mg', 'org', 'ss', 'yc'}
    for model_name in models:
        site_df = pd.DataFrame()
        for dd in days:
            model_file_dir = f'./DATA/MODEL/{model_name}/{yyyymm}/{dd}/'
            csv_files = glob.glob(f'{model_file_dir}/*.csv')
            
            # 찾은 CSV 파일들을 출력
            for csv_file in csv_files:
                model_road = pd.read_csv(csv_file)
                cond1 = model_road['road_name'] == road_name
                cond2 = model_road['direction'].str.startswith(road_dir_1)
                model_road = model_road[cond1&cond2]
                #site_list.append(model_road)
            site_df = pd.concat([site_df,model_road])

        site_df['date_time'] = pd.to_datetime(site_df['date_time'])
        site_df['hour'] = site_df['date_time'].dt.hour
        site_df['day'] = site_df['date_time'].dt.day
        model_1 = site_df

        model_data_df = model_1.copy() # 해당 조건의 모델 데이터 / 재사용을 위해 복사본 사용
        obs_date_df = data_obs_test.copy() # 해당 조건의 관측 데이터 / 재사용을 위해 복사본 사용

        print('모델 데이터의 컬럼 : ',model_data_df.columns)
        print('관측 데이터의 컬럼 : ',obs_date_df.columns)
        #KDTree 를 통한 근접 점 찾기
        road_df = pd.DataFrame()
        # 첫 번째 줄의 위경도 데이터
        observ_line = np.array(list(zip(obs_date_df['longitude'], obs_date_df['latitude'])))

        # 두 번째 줄의 위경도 데이터
        model_line = np.array(list(zip(model_data_df['lon'], model_data_df['lat'])))

        # KDTree 객체 생성
        tree = KDTree(observ_line)

        # 각 점마다 가장 가까운 점을 찾아 매칭
        matched_points = []
        for point in model_line:
            _, index = tree.query([point], k=1)  # k=1로 설정하여 가장 가까운 점 하나만 선택
            matched_points.append(observ_line[index[0]])

        matched_lon = [point[0][0] for point in matched_points]
        matched_lat = [point[0][1] for point in matched_points]

        df = pd.DataFrame({'lon': model_line[:, 0], 'lat': model_line[:, 1], 'longitude': matched_lon, 'latitude': matched_lat})

        days = model_data_df['day'].unique()

        

        model_date_2 = model_data_df.copy()

        total_df = pd.merge(df,data_obs_test,on=['longitude','latitude'])
        total_df = pd.merge(total_df,model_date_2,on=['lon','lat','hour','day'])
        total_df = total_df.drop_duplicates()
        road_df = pd.concat([road_df,total_df])

        val_data = road_df.drop_duplicates(subset=['lon', 'lat'])
        val_data.to_csv(f"{road_name}{date}{model_name}{road_dir_1}.csv", index=False)
        del val_data

CSV 파일 목록:
./DATA/20230705_스시2_서울&시흥_1월관측_후처리자료\seoul_1_gangbyeon_vaisala_20230119_U.csv
gangbyeon 20230119 U
해당 관측 자료에 포함되는 일 [19]
해당 관측 자료에 포함되는 시간 [15]
관측 자료 개수 31977
모델 데이터의 컬럼 :  Index(['date_time', 'update_time', 'loc', 'lon', 'lat', 'seq', 'p_hour',
       'road_name', 'direction', 'altitude', 'link_id', 'road_temp',
       'road_hydro', 'road_ice', 'hour', 'day'],
      dtype='object')
관측 데이터의 컬럼 :  Index(['timestamp', 'longitude', 'latitude', 'relative_humidity',
       'surface_temperature', 'ice_layer_thickness', 'surface_state',
       'dew_point_temperature', 'water_layer_thickness',
       'frost_point_temperature', 'ambient_temperature', 'grip',
       'snow_layer_thickness', 'year', 'month', 'day', 'hour'],
      dtype='object')
모델 데이터의 컬럼 :  Index(['date_time', 'update_time', 'loc', 'lon', 'lat', 'seq', 'p_hour',
       'road_name', 'direction', 'altitude', 'link_id', 'road_temp',
       'road_hydro', 'road_ice', 'hour', 'day'],
      dtype='object')
관측 데이터의 컬럼 :  Inde

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from sklearn.neighbors import KDTree

import os
import glob
# ver2

# 사용할 관측 데이터

# 디렉토리 내의 모든 CSV 파일 목록 가져오기
csv_files = glob.glob('./DATA/20230705_스시2_서울&시흥_1월관측_후처리자료/' + '*.csv')
print("CSV 파일 목록:")
csv_files = csv_files[-4:]
for file in csv_files:

    print(file)
    obs_df = pd.read_csv(file, low_memory=False)
    obs_df.columns = obs_df.columns.str.lower()

    file_info = file.split('_')

    road_name = file_info[-4]
    date = file_info[-2]
    road_dir_1 = file_info[-1]
    road_dir_1 = road_dir_1.split('.')[0]
    date = file_info[-2]
    print(road_name, date, road_dir_1)

    data_obs_test = obs_df
    #데이터의 timestmp는 센서에 따라서 다른 경우가 있기 때문에 각각 int로 저장해둔 시간 변수를 사용
    data_obs_test["timestamp"] = pd.to_datetime(data_obs_test["timestamp"])
    data_obs_test["year"] = data_obs_test["timestamp"].dt.year
    data_obs_test["month"] = data_obs_test["timestamp"].dt.month
    data_obs_test["day"] = data_obs_test["timestamp"].dt.day
    data_obs_test["hour"] = data_obs_test["timestamp"].dt.hour
    print('해당 관측 자료에 포함되는 일',data_obs_test['day'].unique())
    print('해당 관측 자료에 포함되는 시간',data_obs_test['hour'].unique())
    # 1월 19일 15시 데이터로 진행 > 관측자료
    print("관측 자료 개수",len(data_obs_test))

    day = data_obs_test['day'].unique()[0]
    hour = data_obs_test['hour'].unique()[0]
    year = data_obs_test['year'].unique()[0]
    month = data_obs_test['month'].unique()[0]

    yyyymm = f'{year}{month:02d}'
    dd_obs = day
    hour = hour
    days = [dd_obs-1,dd_obs] # 모델 자료 시간을 KST로 변환으로 인해 한 날짜의 데이터가 2개의 파일에 분할되어 있음
    models = {'jr', 'mg', 'org', 'ss', 'yc'}
    for model_name in models:
        site_df = pd.DataFrame()
        for dd in days:
            model_file_dir = f'./DATA/MODEL/siheung/{model_name}/{yyyymm}/{dd}/'
            csv_files = glob.glob(f'{model_file_dir}/*.csv')
            
            # 찾은 CSV 파일들을 출력
            for csv_file in csv_files:
                model_road = pd.read_csv(csv_file)
                cond1 = model_road['road_name'] == road_name
                cond2 = model_road['direction'].str.startswith(road_dir_1)
                model_road = model_road[cond1&cond2]
                #site_list.append(model_road)
            site_df = pd.concat([site_df,model_road])

        site_df['date_time'] = pd.to_datetime(site_df['date_time'])
        site_df['hour'] = site_df['date_time'].dt.hour
        site_df['day'] = site_df['date_time'].dt.day
        model_1 = site_df

        model_data_df = model_1.copy() # 해당 조건의 모델 데이터 / 재사용을 위해 복사본 사용
        obs_date_df = data_obs_test.copy() # 해당 조건의 관측 데이터 / 재사용을 위해 복사본 사용

        print('모델 데이터의 컬럼 : ',model_data_df.columns)
        print('관측 데이터의 컬럼 : ',obs_date_df.columns)
        #KDTree 를 통한 근접 점 찾기
        road_df = pd.DataFrame()
        # 첫 번째 줄의 위경도 데이터
        observ_line = np.array(list(zip(obs_date_df['longitude'], obs_date_df['latitude'])))

        # 두 번째 줄의 위경도 데이터
        model_line = np.array(list(zip(model_data_df['lon'], model_data_df['lat'])))

        # KDTree 객체 생성
        tree = KDTree(observ_line)

        # 각 점마다 가장 가까운 점을 찾아 매칭
        matched_points = []
        for point in model_line:
            _, index = tree.query([point], k=1)  # k=1로 설정하여 가장 가까운 점 하나만 선택
            matched_points.append(observ_line[index[0]])

        matched_lon = [point[0][0] for point in matched_points]
        matched_lat = [point[0][1] for point in matched_points]

        df = pd.DataFrame({'lon': model_line[:, 0], 'lat': model_line[:, 1], 'longitude': matched_lon, 'latitude': matched_lat})

        days = model_data_df['day'].unique()

        

        model_date_2 = model_data_df.copy()

        total_df = pd.merge(df,data_obs_test,on=['longitude','latitude'])
        total_df = pd.merge(total_df,model_date_2,on=['lon','lat','hour','day'])
        total_df = total_df.drop_duplicates()
        road_df = pd.concat([road_df,total_df])

        val_data = road_df.drop_duplicates(subset=['lon', 'lat'])
        val_data.to_csv(f"{road_name}{date}{model_name}{road_dir_1}.csv", index=False)
        del val_data

CSV 파일 목록:
./DATA/20230705_스시2_서울&시흥_1월관측_후처리자료\sihueng_SHR1_lufft_20230120_D.csv
SHR1 20230120 D
해당 관측 자료에 포함되는 일 [20]
해당 관측 자료에 포함되는 시간 [1]
관측 자료 개수 4598
모델 데이터의 컬럼 :  Index(['date_time', 'update_time', 'loc', 'lon', 'lat', 'seq', 'p_hour',
       'road_name', 'direction', 'altitude', 'link_id', 'road_temp',
       'road_hydro', 'road_ice', 'hour', 'day'],
      dtype='object')
관측 데이터의 컬럼 :  Index(['timestamp', 'road temperature100 [°c] cur',
       'amb. temperature110 [°c] cur', 'dewpoint temperature120 [°c] cur',
       'rel. humidity o.r.200 [%] cur', 'rel. humidity210 [%] cur',
       'waterfilm height610 [mm] cur', 'wfh on surface611 [mm] cur',
       'snow height612 [mm] cur', 'ice percentage800 [%] cur',
       'friction820 [n/a] cur', 'road condition900 [logic] cur', 'image',
       'latitude', 'longitude', 'year', 'month', 'day', 'hour'],
      dtype='object')
모델 데이터의 컬럼 :  Index(['date_time', 'update_time', 'loc', 'lon', 'lat', 'seq', 'p_hour',
       'road_name', 'directi